In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
from pygame import mixer
import tempfile
from gtts import gTTS
import requests
import pygame

pygame 2.1.3 (SDL 2.0.22, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
# url = "http://192.168.0.118:80" #cilab
url = "http://172.20.10.12:80" #phone


In [3]:
# 計算角度的副函式
def calculateAngle(landmark1, landmark2, landmark3):
    # 獲取所需座標
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    # 計算三點之間的夾角
    angle = math.floor(math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2)))
    if angle < 0:
        angle += 360
    if angle>180:
        angle=360-angle
    return angle

In [4]:
# def calculatey(landmark1, landmark2):
#     # 獲取所需座標
#     x1, y1, _ = landmark1
#     x2, y2, _ = landmark2

#     y= abs(math.floor(y1 - y2))

#     return y

In [5]:
# 計算角度的副函式
def calculatexang(landmark1, landmark2):
    # 獲取所需座標
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2

    angle=math.floor(math.degrees(math.atan2(y1 - y2, x1 - x2)))
    if angle < 0:
        angle += 360
    if angle>180:
        angle=360-angle
    return angle

In [6]:
# Text to Speech
def speak(sentence, lang):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts=gTTS(text=sentence, lang=lang)
        tts.save('{}.mp3'.format(fp.name))
        mixer.init()
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play(1)

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    height, width, _ = image.shape
    landmarks = []
    if results.pose_landmarks:
            # 畫關鍵點在圖片上
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS)
        for landmark in results.pose_landmarks.landmark:
        # 將關鍵點加進list內.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                    (landmark.z * width)))    
    try:
        l_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    except:
        l_elbow=0
    try:
        # 取得右肩、右肘和右腕之間的夾角。    # 12 14 16 
        r_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    except:
        r_elbow=0
    try:
        # 取得左肘、左肩和左臀之間的夾角。    # 13 11 23
        l_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
    except:
        l_shoulder=0
    try:
        # 取得右臀、右肩和右肘之間的夾角。    # 14 12 24
        r_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    except:
        r_shoulder=0
    try: # 左臀角度 
        l_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])       
    except:
        l_hip=0
    try: # 右臀角度
        r_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])           
    except:
        r_hip=0
    try:
        # 取得左臀、左膝蓋和左腳踝之間的角度。# 23 25 27
        l_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    except:
        l_knee=0        
    try:
        # 取得右臀、右膝蓋和右腳踝之間的角度。# 24 26 28 
        r_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    except:
        r_knee=0
    try:
        # 取得左臀、左膝蓋和左腳踝之間的角度。# 23 25 27
        l_knee2ankle = calculatexang( landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    except:
        l_knee2ankle=0        
    try:
        # 取得右臀、右膝蓋和右腳踝之間的角度。# 24 26 28 
        r_knee2ankle = calculatexang( landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    except:
        r_knee2ankle=0
    return landmarks,image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle

In [8]:
def test_http():
    try:
        response = requests.get(url, timeout=5) # 發送 GET 請求
        if response.status_code == 200:
            x=1
            print("伺服器運作正常")
            return x
        else:
            print(f"伺服器回傳了錯誤的狀態碼: {response.status_code}")
            x=0
            return x
    except :
        pass

In [9]:
def touch(mp3name):
    pygame.mixer.init()
    pygame.mixer.music.load(mp3name)
        # 如果按下 q 鍵，就停止處理影像
    pygame.mixer.music.play()
    while mixer.music.get_busy() == True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            pygame.mixer.music.stop()
        # cv2.destroyAllWindows()    
    # 停止播放音樂
    pygame.mixer.music.stop()
    

In [10]:
def main(mp3name,cap):
    pygame.mixer.init()
    pygame.mixer.music.load(mp3name)
    

    with mp_pose.Pose(static_image_mode=False, 
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5, 
                        model_complexity=0)as pose:
    # 處理攝像頭影像
        while cap.isOpened():
        # while ret:
            pygame.mixer.music.play()
            while(pygame.mixer.music.get_busy()):
                ret, frame = cap.read()
                landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

                color=(0,0,255)
                try:
                        cv2.putText(output_image,str(r_elbow),
                                (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
                except:
                    pass
                try:
                        cv2.putText(output_image,str(r_shoulder),
                                (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
                except:
                    pass
                try:
                        cv2.putText(output_image,str(r_hip),
                                (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
                except:
                    pass
                try:
                        cv2.putText(output_image,str(r_knee),
                                (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
                except:
                    pass
                cv2.putText(output_image,str(mp3name),(20,25),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA,)
                # 顯示影像
                cv2.imshow('OpenCV Feed', output_image)

                # 如果按下 q 鍵，就停止處理影像
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    pygame.mixer.music.stop()
                    # cv2.destroyAllWindows()    
                    break
                # 釋放資源

            # 停止播放音樂
            pygame.mixer.music.stop()
            break
    

In [11]:
# pose 2 前彎向下
def pose2(cap,x):
    lastTime1 = 0
    lastTime2 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while (cap.isOpened()):
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA =l_hip>60 or r_hip>60
            condB = l_knee<150 or r_knee<150
                
            if condA and (time()-lastTime1)<4:
                text1= 'upper body close to thigh'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'upper body close to thigh'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("上身貼近大腿",'zh-tw')
                lastTime1 = time()
                if(x==1):
                    data = {'shoulder': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                count=count+1
                print (count)
                if (count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("雙腿盡量伸直",'zh-tw')
                    lastTime2 = time() 
                    if(x==1):
                        data = {'knee': '1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                    count=count+1
                    print(count)
                    if(count>5):
                        speak("那我們再聽一次",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue    
                        return count            
                else:
                    speak("太棒了",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    count=0
                    return count
            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()
                count=0    
                return count
            
        # cap.release()
        # cv2.destroyAllWindows()

In [12]:
# pose 3 背部向前延伸
def pose3(cap,x):
    lastTime1 = 0
    lastTime2 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        # speak("待會要做前彎向下,請注意平衡安全__  ",'zh-tw')   
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA =l_hip<50 or r_hip<50
            condB = l_knee<160 or r_knee<160
                
            if condA and (time()-lastTime1)<4:
                text1= 'back forward'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'back forward'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("上身抬高,背部向前延伸",'zh-tw')
                if(x==1):
                    data = {'shoulder': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                lastTime1 = time()
                count=count+1
                print (count)
                if(count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("雙腿盡量伸直",'zh-tw')
                    lastTime2 = time() 
                    if(x==1):
                        data = {'knee': '1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                    count=count+1
                    print (count)
                    if(count>5):
                        speak("那我們再聽一次",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        return count
                    
                else:
                    speak("太棒了",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    count=0
                    return count 
            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()
                count=0    
                return count

In [13]:
# pose 4-1 四足跪姿
def pose4_1(cap,x):
    lastTime1 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA = l_knee2ankle>30 or r_knee2ankle>30
 
                    
            if condA and (time()-lastTime1)<6:
                text1= 'legs back'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=6 :
                text1= 'legs back'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("雙腳往後，膝蓋貼在地板上",'zh-tw')
                lastTime1 = time()
                if(x==1):
                    data = {'knee': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                count=count+1
                print(count)
                if(count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:
                text1= ' success '
                cv2.putText(output_image, text1, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                
                count=0
                return count
            

            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()
                count=0
                return count

In [14]:
# pose 4 下犬式
def pose4_2(cap,x):
    lastTime1 = 0
    lastTime2 = 0
    lastTime3 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA = l_knee<145 or r_knee<145
            condB = l_shoulder<145 or r_shoulder<145
            condC = l_elbow<145 or r_elbow<145  
                    
            if condA and (time()-lastTime1)<4:
                text1= 'legs straight'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'legs straight'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("雙腳伸直",'zh-tw')
                lastTime1 = time()
                if(x==1):
                    data = {'knee': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                count=count+1
                print(count)
                if(count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<5 :
                    text2= 'shoulder and back straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=5 :
                    text2= 'shoulder and back straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("背部打直,背部與手臂成一直線",'zh-tw')
                    lastTime2 = time()
                    if(x==1):
                        data = {'shoulder': '1'}
                        response = requests.post(url, data=data)
                        print(response.text) 
                    count=count+1
                    print(count)
                    if(count>5):
                        speak("那我們再聽一次",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        return count
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<4 :
                        text3= 'elbows straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condC and (time()-lastTime3)>=4 :
                        text3= 'elbows straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("雙手打直",'zh-tw')
                        lastTime3 = time() 
                        if(x==1):
                            data = {'r_elbow': '1'}
                            response = requests.post(url, data=data)
                            print(response.text)
                        count=count+1
                        print(count)
                        if(count>5):
                            speak("那我們再聽一次",'zh-tw')
                            while mixer.music.get_busy() == True:
                                continue
                            return count
                    else:
                        text3= ' success '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("太棒了",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        count=0
                        return count

            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()
                count=0
                return count

In [15]:
# pose 5 平板式
def pose5(cap,x):
    lastTime1 = 0
    lastTime2 = 0
    lastTime3 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA = l_shoulder>100 or r_shoulder>100
            condB = l_knee <150 or r_knee <150  
            condC = l_hip <150 or r_hip <150  

            if condA and (time()-lastTime1)<4:
                text1= 'Shoulders come directly over wrists'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'Shoulders come directly over wrists'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("肩膀來到手腕的正上方",'zh-tw')
                lastTime1 = time()
                if(x==1):
                    data = {'shoulder': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                count=count+1
                print(count)
                if(count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'hip and leg straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'ship and leg straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("膝蓋打直",'zh-tw')
                    lastTime2 = time()
                    if(x==1):
                        data = {'knee': '1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                    count=count+1
                    print(count)
                    if(count>5):
                        speak("那我們再聽一次",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        return count
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<4 :
                        text3= 'hip straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condC and (time()-lastTime3)>=4 :
                        text3= 'hip straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("臀部打平",'zh-tw')
                        lastTime3 = time()
                        if(x==1):
                            data = {'hips': '1'}
                            response = requests.post(url, data=data)
                            print(response.text) 
                        count=count+1
                        print(count)
                        if(count>5):
                            speak("那我們再聽一次",'zh-tw')
                            while mixer.music.get_busy() == True:
                                continue
                            return count
                    else:
                        text2= ' success '
                        cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("太棒了",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        count=0
                        return count

            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()    
                count=0    
                return count

In [16]:
# pose 6 眼鏡蛇式
def pose6(cap,x):
    lastTime1 = 0
    lastTime2 = 0
    lastTime3 = 0
    count=0
    # cap = cv2.VideoCapture(cap_name)  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee,l_knee2ankle,r_knee2ankle=mediapipe_detection(frame, pose)

            condA = l_knee<160 or r_knee<160
            condB = l_hip >150 or r_hip >150
            condC = l_shoulder>50 or r_shoulder>50

            if condA and (time()-lastTime1)<4:
                text1= 'straight the legs'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'straight the legs'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("雙腿伸直",'zh-tw')
                lastTime1=time()
                if(x==1):
                    data = {'knee': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                count=count+1
                print (count)
                if(count>5):
                    speak("那我們再聽一次",'zh-tw')
                    while mixer.music.get_busy() == True:
                        continue
                    return count
            else:
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'low-body on the floor'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'low-body on the floor'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("下半身貼好在地板上",'zh-tw')
                    lastTime2 = time() 
                    if(x==1):
                        data = {'hips': '1'}
                        response = requests.post(url, data=data)
                        print(response.text) 
                    count=count+1
                    print (count)
                    if(count>5):
                        speak("那我們再聽一次",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        return count 
                else:    
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<4 :
                        text3= 'upper body improvement '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condC and (time()-lastTime3)>=4 :
                        text3= 'upper body improvement '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("上半身提高",'zh-tw')
                        lastTime3 = time() 
                        if(x==1):
                            data = {'shoulder': '1'}
                            response = requests.post(url, data=data)
                            print(response.text) 
                        count=count+1
                        print (count)
                        if(count>5):
                            return count
                    else:
                        text3= ' success '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("太棒了",'zh-tw')
                        while mixer.music.get_busy() == True:
                            continue
                        count=0
                        return count
            color=(0,0,255)
            try:
                cv2.putText(output_image,str(r_elbow),
                            (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_shoulder),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
            try:
                cv2.putText(output_image,str(r_hip),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass
        
            try:
                cv2.putText(output_image,str(r_knee),
                        (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],),cv2.FONT_HERSHEY_SIMPLEX,1,color,2,cv2.LINE_AA,)
            except:
                pass

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                # cv2.destroyAllWindows()    
                count=0
                return count


In [17]:
def test_vibration(x):
    try:
            if(x==1):
                data = {'elbows': '1'}
                response = requests.post(url, data=data)
                print(response.text)
            elif(x==2):
                data = {'shoulder': '1'}
                response = requests.post(url, data=data)
                print(response.text)
            elif(x==3):
                data = {'hips': '1'}
                response = requests.post(url, data=data)
                print(response.text)
            elif(x==4):
                data = {'knee': '1'}
                response = requests.post(url, data=data)
                print(response.text)
    except:
         pass
    
    

In [18]:
def pre(a):
    if(a==1):
        speak("前彎向下介紹",'zh-tw')
        while mixer.music.get_busy() == True:
            continue
        touch("pose2_cut.mp3")
    elif(a==2):
        speak("下犬式介紹",'zh-tw')
        while mixer.music.get_busy() == True:
            continue
        touch("pose4_cut.mp3")
    elif(a==3):
        speak("平板式介紹",'zh-tw')
        while mixer.music.get_busy() == True:
            continue
        touch("pose5_cut.mp3")
    elif(a==4):
        speak("眼鏡蛇式介紹",'zh-tw')
        while mixer.music.get_busy() == True:
            continue
        touch("pose6_cut.mp3")

In [19]:
x=test_http()
if x==1:
    test_vibration(1)

In [53]:
pre(1) #1前彎#2下犬#3平板#4眼鏡蛇
pre(2)
pre(3)
pre(4)

In [20]:
cap = cv2.VideoCapture("yoga-1.mp4") # 開啟鏡頭
x=test_http() # 測試http狀態

main("part1.mp3",cap)
count=10
while(count>5):
    main("part2.mp3",cap) 
    count=pose2(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part3.mp3",cap)
    count=pose3(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part4-1.mp3",cap)
    count=pose4_1(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part4-2.mp3",cap)
    count=pose4_2(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part5.mp3",cap)
    count=pose5(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part6.mp3",cap)
    count=pose6(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part7.mp3",cap)
    count=pose4_2(cap,x)
    if count==0:
        break
count=10
while(count>5):
    main("part8.mp3",cap)
    count=pose2(cap,x)
    if count==0:
        break
main("part9.mp3",cap)
cap.release()
cv2.destroyAllWindows()

1
1
1
2
3
4
5
6
1
2
1
1
1
1
